# Sentiment140

In [2]:
%%bash 

wget "http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip"
unzip trainingandtestdata.zip -d data/
iconv -f iso-8859-1 -t utf-8 < "data/training.1600000.processed.noemoticon.csv" > "data/sentiment140data.csv"
cat data/sentiment140data.csv | perl -pe 's/\"\,\"/\"\t\"/g' > data/sentiment140.csv


Archive:  trainingandtestdata.zip
  inflating: data/testdata.manual.2009.06.14.csv  
  inflating: data/training.1600000.processed.noemoticon.csv  


URL transformed to HTTPS due to an HSTS policy
--2019-04-25 10:56:04--  https://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
Resolving cs.stanford.edu (cs.stanford.edu)... 171.64.64.64
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81363704 (78M) [application/zip]
Saving to: ‘trainingandtestdata.zip.3’

     0K .......... .......... .......... .......... ..........  0%  130K 10m12s
    50K .......... .......... .......... .......... ..........  0%  180M 5m6s
   100K .......... .......... .......... .......... ..........  0%  259K 5m6s
... 79250K .......... .......... .......... .......... .......... 99% 97.0M 0s
 79300K .......... .......... .......... .......... .......... 99% 67.9M 0s
 79350K .......... .......... .......... .......... .......... 99%  142M 0s
 79400K .......... .......... .......... .......... .......... 99% 67.3M 0s
 79450K ......                                        

In [4]:
%%bash

cat data/sentiment140.csv | head -2

"0"	"1467810369"	"Mon Apr 06 22:19:45 PDT 2009"	"NO_QUERY"	"_TheSpecialOne_"	"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"
"0"	"1467810672"	"Mon Apr 06 22:19:49 PDT 2009"	"NO_QUERY"	"scotthamilton"	"is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!"


# Amazon

In [1]:
%%bash

#wget "http://www.cs.jhu.edu/~mdredze/datasets/sentiment/unprocessed.tar.gz"
tar -ztvf unprocessed.tar.gz 
tar -zxvf unprocessed.tar.gz sorted_data/
#iconv -f iso-8859-1 -t utf-8 < sorted_data/apparel/all.review > review_apparel.txt
#mv review_apparel.txt data/


drwxr-xr-x blitzer/blitzer   0 2009-03-09 05:57 sorted_data/
drwxr-xr-x blitzer/blitzer   0 2007-05-07 01:29 sorted_data/dvd/
-rwxr-xr-x blitzer/blitzer 34488889 2007-05-07 01:29 sorted_data/dvd/processed.review.balanced
-rwxr-xr-x blitzer/blitzer 125103795 2007-05-06 23:27 sorted_data/dvd/processed.review
-rwxr-xr-x blitzer/blitzer   1684389 2007-05-04 22:22 sorted_data/dvd/negative.review
-rwxr-xr-x blitzer/blitzer 204186281 2007-05-04 22:23 sorted_data/dvd/unlabeled.review
-rwxr-xr-x blitzer/blitzer   1701174 2007-05-04 22:23 sorted_data/dvd/positive.review
-rwxr-xr-x blitzer/blitzer 207571844 2007-05-06 21:21 sorted_data/dvd/all.review
drwxr-xr-x blitzer/blitzer         0 2007-05-07 01:27 sorted_data/baby/
-rwxr-xr-x blitzer/blitzer   2819906 2007-05-07 01:27 sorted_data/baby/processed.review.balanced
-rwxr-xr-x blitzer/blitzer   6580913 2007-05-06 21:34 sorted_data/baby/processed.review
-rwxr-xr-x blitzer/blitzer    929961 2007-05-04 22:23 sorted_data/baby/negative.review
-rwxr-xr

In [5]:
%%writefile review_preprocessing.py

#This script converts review_apparel.txt into a .csv file with a similar format as sentiment140.csv

sentiment = []
review = []
previous = ""


f = open("data/review_apparel.txt", "r")
for line in f:
    if previous == '<rating>':
        if line.strip() == "1.0" or line.strip() == "2.0":
            sentiment.append("0")
        else:
            sentiment.append("4")
        previous = line.strip()
    elif previous.strip() == "<review_text>":
        review.append(line)
        previous = line.strip()
    else: 
        previous = line.strip()

f.close()

data = list(zip(sentiment, review))

for i in data: 
    print(i[0] + "\t" + i[1])

Overwriting review_preprocessing.py


In [13]:
%%bash 

python3 review_preprocessing.py | egrep -v '^$' > reviews.csv

In [14]:
%%bash

cat ../data/reviews.csv | head -2

0	I want to start by saying Fred Flare- shipped this product very fast!! And the transaction itself was very smooth. I do however, have extreme problems with the product itself. The product is not leather, its nylon, and it sort of looks cheap? The inside material is sued, but that's only the lining for the base of the wallet. Also, The wallet part is very hard to use. You cant really put too much in the wallet- The credit card slots are a little too snug, and there is no place for my I.D. The wallet included a small "note book" but it also doesn't fit in the wallet? I was very excited about this product, but now I feel duped. The pictures made the wallet seem like it was of higher quality, and that it was user friendly, but it's not. I do not recommend this product
0	I have to say that I was disappointed when I opened up the package containing my iPod wallet. It's cute, but not $60ish cute. First of all, it's not leather, it's nylon. The lining is indeed suede, but the photos in the p

In [25]:
%%writefile balancing_data.py

#Balancing data

import random

f = open("data/reviews.csv", "r")
all_positives = []
negatives = []

for line in f: 
    line = line.strip()
    both = line.split("\t")
    if both[0] == "0":
        if len(both) == 2:
            negatives.append(both)
    else: 
        if len(both) == 2:
            all_positives.append(both)
f.close()

random.seed(1234)
positives = random.sample(all_positives, 150120)
 #len(negatives)=150120
balanced_data = positives + negatives

for i in balanced_data:
    print(i[0] + "\t" + i[1])

Overwriting balancing_data.py


In [26]:
%%bash 

python3 balancing_data.py > reviews_balanced.csv